In [1]:
import pandas as pd
import random
import sys
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import random_swaps
from model.amm.global_state import GlobalState, swap, add_liquidity, migrate, migrate_lp, create_sub_pool
from model.amm import omnipool_amm as oamm

# same seed, same parameters = same simulation result
random.seed(42)

# set up the state

initial_state = GlobalState(
    pools = {
        "Omnipool": OmnipoolState(
            tokens={
                'HDX': {'liquidity': 1000000, 'LRNA_price': 4},
                'USD': {'liquidity': 1000000, 'LRNA_price': 100},  # this means 100 LRNA/ 1 USD.
                'USDA': {'liquidity': 1000000, 'LRNA_price': 100},  # alternatively, you can specify 'LRNA': 100000000
                'USDB': {'liquidity': 1000000, 'LRNA_price': 100},  # same result.
                'USDC': {'liquidity': 1000000, 'LRNA_price': 100},
                'DAI': {'liquidity': 1000000, 'LRNA_price': 100},
                'R1': {'liquidity': 1000000, 'LRNA_price': 50},
                'R2': {'liquidity': 3000000, 'LRNA_price': 150},
            },
            lrna_fee=0,  # these can also be FeeMechanism type
            asset_fee=0
        )
    },
    agents = {
        'Trader': Agent(
            holdings={'HDX': 10000, 'USD': 10000, 'R1': 10000, 'R2': 10000},
        ),
        'LP': Agent(
            holdings={'DAI': 1000000}, 
        )
    }
).create_sub_pool(
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    tkns_migrate=['USDA', 'USDB', 'USDC'], 
    amplification=10,
    trade_fee=0
)
print(initial_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0%
    asset fee: 0%
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 4000000.0
        USD price: 0.04
        tvl: 40000.0
        weight: 4000000.0/1004000000.0 (0.00398406374501992)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 100000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 100000000.0/1004000000.0 (0.099601593625498)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *DAI*
        asset quantity: 1000000.0
        lrna quantity: 100000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 100000000.0/1004000000.0 (0.099601593625498)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1

In [2]:
print(initial_state.pools['Omnipool'].liquidity['Stableswap'])

300000000.0


In [3]:
migrate_state = migrate(
    old_state=initial_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    tkn_migrate='DAI'
)

# sell an asset
buy_quantity = 1000
tkn_buy = 'DAI'
tkn_sell = 'R1'

buy_state = swap(
    old_state=migrate_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    buy_quantity=buy_quantity
)
print(buy_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0%
    asset fee: 0%
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 4000000.0
        USD price: 0.04
        tvl: 40000.0
        weight: 4000000.0/1004000000.0 (0.00398406374501992)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 100000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 100000000.0/1004000000.0 (0.099601593625498)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *R1*
        asset quantity: 1002004.5113238489
        lrna quantity: 49899974.93518265
        USD price: 0.4980014997
        tvl: 498999.74935182655
        weight: 49899974.93518265/1004000000.0 (0.04970117025416599)
        weight cap: 1.0
        total shar

In [4]:
# reverse that operation
sell_quantity = initial_state.agents['Trader'].holdings[tkn_sell] - buy_state.agents['Trader'].holdings[tkn_sell]
sell_state = swap(
    old_state=initial_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    sell_quantity=sell_quantity
)
print(sell_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0%
    asset fee: 0%
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 4000000.0
        USD price: 0.04
        tvl: 40000.0
        weight: 4000000.0/1004000000.0 (0.00398406374501992)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 100000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 100000000.0/1004000000.0 (0.099601593625498)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *DAI*
        asset quantity: 999000.7488534335
        lrna quantity: 100100025.06481734
        USD price: 1.0020015018
        tvl: 1001000.2506481734
        weight: 100100025.06481734/1004000000.0 (0.09970122018408102)
        weight cap: 1.0
        total sh

In [5]:
# chain operations together
partial_lp_state = add_liquidity(
    old_state=sell_state,
    pool_id='Omnipool',
    agent_id='LP',
    tkn_add='USDA',
    quantity=initial_state.agents['LP'].holdings['USDA']
)
migrate_state = migrate(
    old_state=partial_lp_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    tkn_migrate='USDA'
)
migrate_lp_state = migrate_lp(
    old_state=migrate_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    agent_id='LP',
    tkn_migrate='USDA'
)
print('Migrate State:', migrate_state)

print('Final State:', migrate_lp_state)

ZeroDivisionError: 